# Quickstart guide Qumulo API python bindings

This jupyter notebook is intended to provide a quick walkup experience for using the Qumulo API python bindings.

### Setup steps
1. Prerequisites: python 3.7, pip, jupyter-notebook
2. Make sure you have access to a Qumulo cluster as well as api credentials.
3. Install the Qumulo API python library: <code>**pip install qumulo_api**</code>
4. Run <code>**jupyter-notebook**</code> from the location of this github repo on your local machine.

In [4]:
import os
from qumulo.rest_client import RestClient

In [5]:
# set your environment variables or fill in the variables below
API_HOSTNAME = os.environ['API_HOSTNAME'] if 'API_HOSTNAME' in os.environ else '{your-cluster-hostname}'
API_USER =     os.environ['API_USER']     if 'API_USER'     in os.environ else '{api-cluster-user}'
API_PASSWORD = os.environ['API_PASSWORD'] if 'API_PASSWORD' in os.environ else '{api-cluster-password}'

In [6]:
rc = RestClient(API_HOSTNAME, 8000)
rc.login(API_USER, API_PASSWORD)
rc.auth.who_am_i()

{u'home_directory': None,
 u'id': u'500',
 u'name': u'admin',
 u'primary_group': u'513',
 u'sid': u'S-1-5-21-794736294-2450901474-4127211561-500',
 u'uid': u'0'}

### List all cluster's nodes

Show  each node and it's corresponding status and serial number.

In [7]:
all_nodes = rc.cluster.list_nodes()
for node in all_nodes:
    print "Node: %(id)2s/%(node_name)s - Status: %(node_status)s - Serial: %(serial_number)s" % node

Node:  1/product-1 - Status: online - Serial: sCC121500C
Node:  2/product-2 - Status: online - Serial: sCC120Z001
Node:  3/product-3 - Status: online - Serial: sCC120Z004
Node:  4/product-4 - Status: online - Serial: sCC130400F


### Filesystem stats

Shows the usable capacity of the cluster as well as used capacity.

In [8]:
file_system_stats = rc.fs.read_fs_stats()
free_bytes = float(file_system_stats['free_size_bytes'])
usable_bytes = float(file_system_stats['total_size_bytes'])
used_perc = (usable_bytes-free_bytes) / usable_bytes
print("Usable Capacity (TB): %s" % (round(usable_bytes/pow(1000,4), 2), ))
print("Used Capacity   (TB): %s (%s%%)" % (round((usable_bytes-free_bytes)/pow(1000,4), 2), round(100*used_perc,1)))

Usable Capacity (TB): 47.95
Used Capacity   (TB): 27.02 (56.4%)


### Protection details & status

Show the cluster's data protection status and details.

In [9]:
protection_details = rc.cluster.get_protection_status()
data_str = """
                    Type: %(protection_system_type)s
           Encoding size: %(blocks_per_stripe)s/%(data_blocks_per_stripe)s
      Max drive failures: %(max_drive_failures)s
       Max node failures: %(max_node_failures)s
Remaining drive failures: %(remaining_drive_failures)s
 Remaining node failures: %(remaining_node_failures)s
"""
print(data_str % protection_details)


                    Type: PROTECTION_SYSTEM_TYPE_EC
           Encoding size: 6/4
      Max drive failures: 2
       Max node failures: 1
Remaining drive failures: 2
 Remaining node failures: 1



### HDD and SSD drives status

Enumerate each node's SSDs and HDDs as well as their status, size, and model.

In [10]:
all_drives = rc.cluster.get_cluster_slots_status()
node_num = 0
for drive in all_drives:
    if drive['node_id'] != node_num:
        print "-" * 60
        node_num = drive['node_id']
    cap = float(drive['capacity'])/pow(1000,4)
    if cap > 1:
        cap = str(int(cap)) + " TB"
    else:
        cap = str(int(cap * 1001)) + " GB"
    drive['cap'] = cap
    print "%(node_id)4s.%(slot)s - %(slot_type)s - %(state)s - %(cap)6s - %(disk_model)s" % drive

------------------------------------------------------------
   1.1 - SSD - healthy - 800 GB - INTEL_SSDSC2BB800G4
   1.2 - SSD - healthy - 800 GB - INTEL_SSDSC2BB800G4
   1.3 - HDD - healthy -   6 TB - HGST_HUS726060ALA640
   1.4 - HDD - healthy -   6 TB - HGST_HUS726060ALA640
   1.5 - HDD - healthy -   6 TB - HGST_HUS726060ALA640
   1.6 - HDD - healthy -   6 TB - HGST_HUS726060ALA640
------------------------------------------------------------
   2.1 - SSD - healthy - 800 GB - INTEL_SSDSC2BB800G4
   2.2 - SSD - healthy - 800 GB - INTEL_SSDSC2BB800G4
   2.3 - HDD - healthy -   6 TB - HGST_HUS726060ALA640
   2.4 - HDD - healthy -   6 TB - HGST_HUS726060ALA640
   2.5 - HDD - healthy -   6 TB - HGST_HUS726060ALA640
   2.6 - HDD - healthy -   6 TB - HGST_HUS726060ALA640
------------------------------------------------------------
   3.1 - SSD - healthy - 800 GB - INTEL_SSDSC2BB800G4
   3.2 - SSD - healthy - 800 GB - INTEL_SSDSC2BB800G4
   3.3 - HDD - healthy -   6 TB - HGST_HUS726060ALA64